In [8]:
import os
import json
from sentence_transformers import SentenceTransformer

In [10]:
# 파일 위치 지정
keys_file_path = os.path.join('data', 'api_keys.txt')

# 파일에서 API 키를 로드하는 함수
def load_api_keys(file_path):
    with open(file_path, 'r', encoding='utf-8') as file:
        keys = json.load(file)
    return keys

# API 키 사용
api_keys = load_api_keys(keys_file_path)
openAI_keys = api_keys['openAI_keys']
pinecone_key1 = api_keys['pinecone_key1']
pinecone_key2 = api_keys['pinecone_key2']

In [11]:
from openai import OpenAI

client = OpenAI(api_key=openAI_keys)

with open('data/documents.txt', 'r', encoding="UTF-8") as f:
    lines = f.readlines()

model = SentenceTransformer('sentence-transformers/all-mpnet-base-v2')

# 추후 [:5] 제거시 문서 끝까지
embedding_list = []
for line in lines[:5]:
    embedding_list.append(model.encode(line))

C:\Users\Tomas\AppData\Roaming\Python\Python310\site-packages\huggingface_hub\file_download.py:149: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\Tomas\.cache\huggingface\hub\models--sentence-transformers--all-mpnet-base-v2. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


In [12]:
em_vectors = []
for i, em in enumerate(embedding_list):
    em_vectors.append({"id": str(i), "values": em, "metadata": {"assertion": lines[i]}})

In [13]:
from pinecone import Pinecone
pc = Pinecone(api_key=pinecone_key2)

index = pc.Index("mpnet")

index.upsert(
    vectors=em_vectors
)

{'upserted_count': 5}